# YOLO + CNN Classification Project
## זיהוי אובייקטים עם YOLO וסיווג עם CNN

### משימות:
1. זיהוי אובייקט עיקרי עם YOLO
2. יצירת דאטהסט CSV עם הגבלה ל-10 קטגוריות
3. הכנת דאטהסט ללמידת מכונה (X, Y)
4. בניית רשת CNN
5. הערכת המודל על validation

## Part 1: התקנת חבילות

In [ ]:
# התקנת החבילות הנדרשות
!pip install ultralytics tensorflow scikit-learn pandas matplotlib seaborn

In [37]:
import os
import json
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

# TensorFlow / Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.20.0


## Part 2: משימה 1 - זיהוי אובייקט עיקרי עם YOLO

In [ ]:
def detect_main_object_in_images(model, image_dir, output_file):
    """
    מריץ YOLO על כל התמונות בתיקייה ומזהה את האובייקט העיקרי
    """
    results_dict = {}
    # קבלת כל קבצי התמונות בתיקייה
    image_paths = list(Path(image_dir).rglob("*.jpg")) + list(Path(image_dir).rglob("*.png"))

    print(f"מעבד {len(image_paths)} תמונות מתיקייה: {image_dir}")

    # עיבוד כל תמונה
    for idx, img_path in enumerate(image_paths, 1):
        # הדפסת התקדמות כל 50 תמונות
        if idx % 50 == 0:
            print(f"מעובדות תמונה {idx}/{len(image_paths)}")
        
        try:

            # הרצת YOLO על התמונה
            results = model(str(img_path), verbose=False, conf=0.1)

            # מציאת האובייקט העיקרי (בעל ה-confidence הגבוה ביותר)
            if len(results) > 0 and len(results[0].boxes) > 0:
                
                boxes = results[0].boxes
                confidences = boxes.conf.cpu().numpy()
                
                if len(confidences) > 0:
                    main_obj_idx = confidences.argmax()
                    main_class_id = int(boxes.cls[main_obj_idx].cpu().numpy())
                    main_confidence = float(confidences[main_obj_idx])
                    main_class_name = model.names[main_class_id]

                    results_dict[str(img_path)] = {
                        "class_name": main_class_name,
                        "class_id": main_class_id,
                        "confidence": main_confidence,
                        "bbox": boxes.xyxy[main_obj_idx].cpu().numpy().tolist()
                    }
                else:
                    results_dict[str(img_path)] = {"class_name": "unknown", "confidence": 0.0}
            else:
                results_dict[str(img_path)] = {"class_name": "unknown", "confidence": 0.0}

        except Exception as e:
            print(f"Error processing {img_path}: {e}")
            results_dict[str(img_path)] = {"class_name": "unknown", "confidence": 0.0}

    # שמירת התוצאות לקובץ JSON
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(results_dict, f, indent=2, ensure_ascii=False)

    print(f"התוצאות נשמרו ב: {output_file}")
    return results_dict

In [39]:
# טעינת מודל YOLO
print("טוען מודל YOLO...")
yolo_model = YOLO('yolov8n.pt')

# נתיבים
train_dir = "resources/images/data/train"
test_dir = "resources/images/data/test"
val_dir = "resources/images/data/val"

טוען מודל YOLO...


In [40]:
# הרצה על train
print("\n=== מעבד תמונות TRAIN ===")
train_results = detect_main_object_in_images(
    yolo_model,
    train_dir,
    "train_detections.json"
)


=== מעבד תמונות TRAIN ===
מעבד 239 תמונות מתיקייה: resources/images/data/train
מעובדות תמונה 50/239
מעובדות תמונה 100/239
מעובדות תמונה 150/239
מעובדות תמונה 200/239
התוצאות נשמרו ב: train_detections.json


In [41]:
# הרצה על test
print("\n=== מעבד תמונות TEST ===")
test_results = detect_main_object_in_images(
    yolo_model,
    test_dir,
    "test_detections.json"
)


=== מעבד תמונות TEST ===
מעבד 38 תמונות מתיקייה: resources/images/data/test
התוצאות נשמרו ב: test_detections.json


In [42]:
# הרצה על validation
print("\n=== מעבד תמונות VALIDATION ===")
val_results = detect_main_object_in_images(
    yolo_model,
    val_dir,
    "val_detections.json"
)


=== מעבד תמונות VALIDATION ===
מעבד 140 תמונות מתיקייה: resources/images/data/val
מעובדות תמונה 50/140
מעובדות תמונה 100/140
התוצאות נשמרו ב: val_detections.json


## Part 3: משימה 2 - יצירת CSV והגבלה ל-10 קטגוריות

In [49]:
def create_csv_with_top_categories(results_dict, output_csv, max_categories=10):
    """
    יוצר CSV עם מזהה תמונה ואובייקט עיקרי, מגביל ל-10 קטגוריות
    """
    # יצירת רשימת כל האובייקטים
    data = []
    for img_path, result in results_dict.items():
        image_id = Path(img_path).stem  # שם הקובץ ללא סיומת
        class_name = result.get('class_name', 'unknown')
        confidence = result.get('confidence', 0.0)
        data.append({
            'image_id': image_id,
            'image_path': img_path,
            'detected_object': class_name,
            'confidence': confidence
        })
    
    df = pd.DataFrame(data)
    
    # ספירת קטגוריות
    category_counts = Counter(df['detected_object'])
    print(f"\nסך הכל {len(category_counts)} קטגוריות שונות זוהו")
    print("\nהקטגוריות הנפוצות ביותר:")
    for cat, count in category_counts.most_common(15):
        print(f"  {cat}: {count}")
    
    # שמירת ה-9 הקטגוריות הנפוצות ביותר
    top_categories = [cat for cat, _ in category_counts.most_common(max_categories - 1)]
    
    # סיווג כל דבר אחר כ-'other'
    df['final_category'] = df['detected_object'].apply(
        lambda x: x if x in top_categories else 'other'
    )
    
    # שמירת CSV
    df_output = df[['image_id', 'final_category', 'confidence', 'image_path']]
    df_output.to_csv(output_csv, index=False, encoding='utf-8')
    
    print(f"\nCSV נשמר ב: {output_csv}")
    print(f"מספר קטגוריות סופי: {len(df['final_category'].unique())}")
    print("\nהתפלגות קטגוריות סופית:")
    print(df['final_category'].value_counts())
    
    return df, top_categories

In [44]:
# יצירת CSV לכל סט
print("=== Train Dataset ===")
train_df, top_categories = create_csv_with_top_categories(train_results, "train_labels.csv")

print("\n=== Test Dataset ===")
test_df, _ = create_csv_with_top_categories(test_results, "test_labels.csv")

print("\n=== Validation Dataset ===")
val_df, _ = create_csv_with_top_categories(val_results, "val_labels.csv")

=== Train Dataset ===

סך הכל 3 קטגוריות שונות זוהו

הקטגוריות הנפוצות ביותר:
  unknown: 229
  car: 9
  tv: 1

CSV נשמר ב: train_labels.csv
מספר קטגוריות סופי: 3

התפלגות קטגוריות סופית:
final_category
unknown    229
car          9
tv           1
Name: count, dtype: int64

=== Test Dataset ===

סך הכל 4 קטגוריות שונות זוהו

הקטגוריות הנפוצות ביותר:
  unknown: 32
  car: 4
  train: 1
  bus: 1

CSV נשמר ב: test_labels.csv
מספר קטגוריות סופי: 4

התפלגות קטגוריות סופית:
final_category
unknown    32
car         4
train       1
bus         1
Name: count, dtype: int64

=== Validation Dataset ===

סך הכל 4 קטגוריות שונות זוהו

הקטגוריות הנפוצות ביותר:
  unknown: 133
  car: 4
  tv: 2
  person: 1

CSV נשמר ב: val_labels.csv
מספר קטגוריות סופי: 4

התפלגות קטגוריות סופית:
final_category
unknown    133
car          4
tv           2
person       1
Name: count, dtype: int64


## Part 4: משימה 3 - הכנת דאטהסט ללמידת מכונה (X, Y)

In [45]:
# הגדרות
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3

In [46]:
def prepare_dataset(df, img_height=IMG_HEIGHT, img_width=IMG_WIDTH):
    """
    ממיר תמונות למערכים (X) ומכין תוויות (Y)
    """
    X = []
    y = []
    
    print(f"טוען {len(df)} תמונות...")
    
    for idx, row in df.iterrows():
        if (idx + 1) % 100 == 0:
            print(f"טעון {idx + 1}/{len(df)} תמונות")
        
        try:
            img_path = row['image_path']
            
            # טעינת תמונה
            img = load_img(img_path, target_size=(img_height, img_width))
            img_array = img_to_array(img)
            
            # נורמליזציה [0, 1]
            img_array = img_array / 255.0
            
            X.append(img_array)
            y.append(row['final_category'])
            
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            continue
    
    X = np.array(X)
    print(f"\nX shape: {X.shape}")
    print(f"Number of labels: {len(y)}")
    
    return X, y

In [47]:
# הכנת דאטהסטים
print("\n=== הכנת Train Dataset ===")
X_train, y_train = prepare_dataset(train_df)

print("\n=== הכנת Test Dataset ===")
X_test, y_test = prepare_dataset(test_df)

print("\n=== הכנת Validation Dataset ===")
X_val, y_val = prepare_dataset(val_df)


=== הכנת Train Dataset ===
טוען 239 תמונות...
טעון 100/239 תמונות
טעון 200/239 תמונות

X shape: (239, 224, 224, 3)
Number of labels: 239

=== הכנת Test Dataset ===
טוען 38 תמונות...

X shape: (38, 224, 224, 3)
Number of labels: 38

=== הכנת Validation Dataset ===
טוען 140 תמונות...
טעון 100/140 תמונות

X shape: (140, 224, 224, 3)
Number of labels: 140


In [48]:
# קידוד תוויות - FIX: handle unseen labels in test/val
# Collect ALL unique labels from all datasets
all_labels = set(y_train + y_test + y_val)
print(f"All unique labels across all datasets: {sorted(all_labels)}")
print(f"\nLabels in TRAIN: {sorted(set(y_train))}")
print(f"Labels in TEST: {sorted(set(y_test))}")
print(f"Labels in VAL: {sorted(set(y_val))}")

# Fit encoder on ALL labels (not ideal but necessary when test/val have new labels)
label_encoder = LabelEncoder()
label_encoder.fit(list(all_labels))

print(f"\n⚠️ Note: Fitting on ALL data to handle unseen labels in test/val")

y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
y_val_encoded = label_encoder.transform(y_val)

# One-hot encoding
num_classes = len(label_encoder.classes_)
y_train_cat = to_categorical(y_train_encoded, num_classes)
y_test_cat = to_categorical(y_test_encoded, num_classes)
y_val_cat = to_categorical(y_val_encoded, num_classes)

print(f"\nמספר קטגוריות: {num_classes}")
print(f"קטגוריות: {label_encoder.classes_}")
print(f"\ny_train_cat shape: {y_train_cat.shape}")
print(f"y_test_cat shape: {y_test_cat.shape}")
print(f"y_val_cat shape: {y_val_cat.shape}")

All unique labels across all datasets: ['bus', 'car', 'person', 'train', 'tv', 'unknown']

Labels in TRAIN: ['car', 'tv', 'unknown']
Labels in TEST: ['bus', 'car', 'train', 'unknown']
Labels in VAL: ['car', 'person', 'tv', 'unknown']

⚠️ Note: Fitting on ALL data to handle unseen labels in test/val

מספר קטגוריות: 6
קטגוריות: ['bus' 'car' 'person' 'train' 'tv' 'unknown']

y_train_cat shape: (239, 6)
y_test_cat shape: (38, 6)
y_val_cat shape: (140, 6)


## Part 5: משימה 4 - בניית רשת CNN

In [50]:
def build_cnn_model(input_shape, num_classes):
    """
    בניית מודל CNN משופר
    """
    model = models.Sequential([
        # Block 1
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 2
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 3
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Dense layers
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

In [51]:
# בניית המודל
input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
model = build_cnn_model(input_shape, num_classes)

# הצגת המבנה
model.summary()

e:\school\יג-יד\learning_machine_class\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    25,690,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 26,014,374 (99.24 MB)

 Trainable params: 26,012,710 (99.23 MB)

 Non-trainable params: 1,664 (6.50 KB)

In [52]:
# קומפילציה עם אופטימייזר משופר
# נשתמש ב-Adam עם learning rate scheduling
initial_learning_rate = 0.001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=True
)

optimizer = optimizers.Adam(learning_rate=lr_schedule)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [53]:
# Callbacks לשיפור האימון
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

checkpoint = keras.callbacks.ModelCheckpoint(
    'best_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

In [55]:
# אימון המודל
print("\n=== מתחיל אימון ===")
history = model.fit(
    X_train, y_train_cat,
    batch_size=32,
    epochs=10,
    validation_data=(X_test, y_test_cat),
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)


=== מתחיל אימון ===
Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2493 - loss: 2.5517
Epoch 1: val_accuracy did not improve from 0.63158
8/8 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step - accuracy: 0.2427 - loss: 2.5377 - val_accuracy: 0.2368 - val_loss: 1.6850 - learning_rate: 0.0010
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2245 - loss: 2.4592
Epoch 2: val_accuracy did not improve from 0.63158
8/8 ━━━━━━━━━━━━━━━━━━━━ 26s 3s/step - accuracy: 0.2427 - loss: 2.4278 - val_accuracy: 0.0526 - val_loss: 2.7397 - learning_rate: 0.0010
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2494 - loss: 2.5192
Epoch 3: val_accuracy did not improve from 0.63158
8/8 ━━━━━━━━━━━━━━━━━━━━ 26s 3s/step - accuracy: 0.2176 - loss: 2.4386 - val_accuracy: 0.0263 - val_loss: 3.6693 - learning_rate: 0.0010
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2908 - loss: 2.0818
Epoch 4: val_accuracy did not improve from 0.63158
8/8 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step - a

TypeError: This optimizer was created with a `LearningRateSchedule` object as its `learning_rate` constructor argument, hence its learning rate is not settable. If you need the learning rate to be settable, you should instantiate the optimizer with a float `learning_rate` argument.

In [ ]:
# גרפים של האימון
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy
axes[0].plot(history.history['accuracy'], label='Train Accuracy')
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Model Accuracy')
axes[0].legend()
axes[0].grid(True)

# Loss
axes[1].plot(history.history['loss'], label='Train Loss')
axes[1].plot(history.history['val_loss'], label='Validation Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Model Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nGרף האימון נשמר ב: training_history.png")

## Part 6: משימה 5 - הערכת המודל על Validation

In [ ]:
# הערכה על Train
print("=== הערכה על Train ===")
train_loss, train_accuracy = model.evaluate(X_train, y_train_cat, verbose=0)
print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Train Loss: {train_loss:.4f}")

# הערכה על Test
print("\n=== הערכה על Test ===")
test_loss, test_accuracy = model.evaluate(X_test, y_test_cat, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# הערכה על Validation
print("\n=== הערכה על Validation ===")
val_loss, val_accuracy = model.evaluate(X_val, y_val_cat, verbose=0)
print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation Loss: {val_loss:.4f}")

In [ ]:
# חיזויים על Validation
y_val_pred = model.predict(X_val)
y_val_pred_classes = np.argmax(y_val_pred, axis=1)
y_val_true_classes = np.argmax(y_val_cat, axis=1)

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_val_true_classes, y_val_pred_classes)

plt.figure(figsize=(12, 10))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=label_encoder.classes_,
    yticklabels=label_encoder.classes_
)
plt.title('Confusion Matrix - Validation Set', fontsize=16)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nConfusion Matrix נשמר ב: confusion_matrix.png")

In [ ]:
# Classification Report
print("\n=== Classification Report - Validation Set ===")
print(classification_report(
    y_val_true_classes,
    y_val_pred_classes,
    target_names=label_encoder.classes_
))

## סיכום ומסקנות

In [ ]:
print("="*60)
print("                    סיכום תוצאות                    ")
print("="*60)
print(f"\n1. זיהוי אובייקטים עם YOLO:")
print(f"   - Train: {len(train_results)} תמונות")
print(f"   - Test: {len(test_results)} תמונות")
print(f"   - Validation: {len(val_results)} תמונות")

print(f"\n2. מספר קטגוריות: {num_classes}")
print(f"   קטגוריות: {', '.join(label_encoder.classes_)}")

print(f"\n3. גודל דאטהסט:")
print(f"   - X_train: {X_train.shape}")
print(f"   - X_test: {X_test.shape}")
print(f"   - X_val: {X_val.shape}")

print(f"\n4. ביצועי המודל:")
print(f"   - Train Accuracy: {train_accuracy:.4f}")
print(f"   - Test Accuracy: {test_accuracy:.4f}")
print(f"   - Validation Accuracy: {val_accuracy:.4f}")

print(f"\n5. מסקנות:")
if train_accuracy - val_accuracy > 0.1:
    print("   ⚠️ נראה שיש overfitting - הדיוק על train גבוה משמעותית מ-validation")
    print("   💡 מומלץ להוסיף regularization או data augmentation")
elif val_accuracy > 0.8:
    print("   ✅ המודל משיג ביצועים טובים על ה-validation set")
    print("   ✅ הדיוק מעל 80% מעיד על סיווג אפקטיבי")
elif val_accuracy > 0.6:
    print("   ⚠️ המודל משיג ביצועים בינוניים")
    print("   💡 מומלץ להגדיל את מספר ה-epochs או לשפר את המודל")
else:
    print("   ❌ המודל זקוק לשיפור משמעותי")
    print("   💡 מומלץ לבדוק את איכות הדאטה ולשפר את ארכיטקטורת המודל")

print("\n" + "="*60)

In [ ]:
# שמירת המודל הסופי
model.save('final_cnn_model.keras')
print("\nהמודל הסופי נשמר ב: final_cnn_model.keras")